In [43]:
from tkinter import *
import os.path 
from datetime import date
import pandas as pd
import numpy as np
import re 

## FUNCTION PURPOSE: Allows the user to record the quantity of music practice that has been achieved for the day (in minutes). 
# Function inputs arg 1: None. 
# Function output 1: Creates/updates the log containing practice information.
# Function output 2: df [pandas DataFrame] --> The updated log as a pandas DataFrame. 

# Create a GUI to log the duration of today's music practice.  
def log_progress():
    
    # Make a class object for the GUI.
    class GUI(): 
        def __init__(self, prompt):
            self.prompt = prompt 
            self.response = ""

            def OK():
                self.response = entry.get()
                root.destroy()

            root = Tk()
            lbl = Label(root, text=self.prompt)
            lbl.pack()
            entry = Entry(root)
            entry.pack()

            b = Button(root, text="OK", width=10, command=OK)
            b.pack()
            mainloop()

    # Create an instance object for the GUI. 
    instance_GUI = GUI("Please enter the number of minutes practiced.")
    minutes_practiced_today = instance_GUI.response
    
    # If there was an accidental text entry, notify the user, and ask for a numerical input. 
    while bool(re.search('[^0-9^.]', minutes_practiced_today)) is True or minutes_practiced_today.count('.') > 1: 
        instance_GUI = GUI("Re-enter answer using only numbers 0-9 and one decimal point.")
        minutes_practiced_today = instance_GUI.response
    hours_practiced_today = float(minutes_practiced_today)/60
    
    # Determine the full file path for our log data.  
    cwd = os.getcwd()  
    log_data_directory = cwd.replace('music-practice-log', 'log-data')
    log_data_path = os.path.join(log_data_directory, 'log.csv')
    
    # Create a list to hold the data. 
    df = []
        
    # Create the log file if it does not already exist. 
    if os.path.isfile(log_data_path) is False: 
        
        # Record the new data within a list.
        today = date.today()
        the_date = today.strftime("%d/%m/%Y")
        d = [
            the_date,
            hours_practiced_today,
            hours_practiced_today
            ]
        
        # Append all the information to our dataframe. 
        df.append(d)
        
        # Convert the array to a pandas dataframe. 
        df = pd.DataFrame(df, columns=['Date (DMY)','Practice time (hours)','Cumulative practice time (hours)'])
    
    elif os.path.isfile(log_data_path) is True: 
    
        # Load in the data. 
        previous_data = pd.read_csv(log_data_path)
            
        # Calculate the cumulative number of hours played. 
        previous_hours = previous_data.iloc[:,1]
        total_hours = sum(previous_hours) + float(hours_practiced_today)
        
        # Record the new data within a list. 
        today = date.today()
        the_date = today.strftime("%d/%m/%Y")
        d = [the_date,
             hours_practiced_today,
             total_hours
            ]
        
        # Add the information to our dataframe. 
        previous_data.loc[len(previous_data)] = d
        df = previous_data
        
    # Save our data.
    df.to_csv(log_data_path, index=False)
    
    return df 

In [ ]:
## FUNCTION PURPOSE: A function to update the local graph, and push it to GitHub. 
# Function input arg 1: 
# Function output 1: 

# Create the graph. 

# Push the graph to GitHub. 


In [ ]:
## FUNCTION PURPOSE: A function to WhatsApp message the user to remind them to practice their instrument. 
# Function input arg 1: 
# Function output 1: 

# Determine whether the user has practiced their instrument recently. 

# Create a list of potential WhatsApp messages.

# Send one of the messages to the user. 